In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
path = 'Data/Cleaned_deathCnt/*.csv'
dic_files = Load_files.load_files(path, -14, -12)
data = dic_files['서울']
data['dailyCnt'] = data['defCnt'].diff().fillna(0)
data

,deathCnt,defCnt,gubun,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,stdDay,dailyCnt
0,0,0,서울,Seoul,0.0,0,0,0,0,0.0,2020-01-20,0.0
1,0,0,서울,Seoul,0.0,0,0,0,0,0.0,2020-01-21,0.0
2,0,0,서울,Seoul,0.0,0,0,0,0,0.0,2020-01-22,0.0
3,0,0,서울,Seoul,0.0,0,0,0,0,0.0,2020-01-23,0.0
4,0,1,서울,Seoul,1.0,0,1,0,1,0.0,2020-01-24,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1214,6471,6162486,서울,Seoul,4329.0,0,0,4371,0,65361.0,2023-05-20,4329.0
1215,6473,6166682,서울,Seoul,4196.0,0,0,4195,1,65406.0,2023-05-21,4196.0
1216,6476,6167953,서울,Seoul,1271.0,0,0,1271,0,65419.0,2023-05-22,1271.0
1217,6477,6173315,서울,Seoul,5362.0,0,0,5359,3,65476.0,2023-05-23,5362.0


In [3]:
learning_rates_list = [1e-3, 1e-4, 1e-5]
patiences_list = [20, 50, 100]
num_layers_list = [2, 4, 8]
batch_sizes_list = [16, 32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25, 0.5]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout]
                        i += 1
                        
print(i)

486


In [4]:
criterion = nn.MSELoss()

input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', 'dailyCnt')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

mape_min = np.inf
best_model = {}

for hyperparameter in tqdm(hyperparameter_dict.values(), desc = '서울_RNN', position = 0):
    lr = hyperparameter[0]
    patience = hyperparameter[1]
    num_layers = hyperparameter[2]
    batch_size = hyperparameter[3]
    hidden_size = hyperparameter[4]
    dropout = hyperparameter[5]
    print('-------------------------------------------------------------------------------------')
    print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
          ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
    x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 800, 60, 1, batch_size, 'mto')
    

    model = RNN(input_size = input_size,
                hidden_size = hidden_size,
                sequence_length = sequence_length,
                num_layers = num_layers, 
                dropout = dropout, 
                device = device).to(device)

    optimizer = Adam(model.parameters(), lr = lr)
    
    loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                  test_loader, 
                                                  model, 
                                                  criterion, 
                                                  optimizer, 
                                                  num_epochs, 
                                                  patience,
                                                  device)
    
    label_y, predicted = predict_mto(model, df, x_ss, y_ms)
    
    if mape_min > mape(label_y+1, predicted):
        mape_min = mape(label_y+1, predicted)
        best_model['RNN'] = [mape_min, model, epoch, lr, patience, num_layers, batch_size, hidden_size, dropout]

서울_RNN:   0%|          | 0/486 [00:00<?, ?it/s]

-------------------------------------------------------------------------------------
lr: 0.001  patience: 20  num_layers: 2  batch_size: 16  hidden_size: 8  dropout: 0.25


  0%|          | 0/10000 [00:00<?, ?it/s]


 Early Stopping / epoch: 43 loss: 0.0014
-------------------------------------------------------------------------------------
lr: 0.001  patience: 20  num_layers: 2  batch_size: 16  hidden_size: 8  dropout: 0.5


  0%|          | 0/10000 [00:00<?, ?it/s]

...


 Early Stopping / epoch: 47 loss: 0.0008
-------------------------------------------------------------------------------------
lr: 0.001  patience: 20  num_layers: 2  batch_size: 16  hidden_size: 32  dropout: 0.25


  0%|          | 0/10000 [00:00<?, ?it/s]


 Early Stopping / epoch: 56 loss: 0.0019
-------------------------------------------------------------------------------------
lr: 0.001  patience: 20  num_layers: 2  batch_size: 16  hidden_size: 32  dropout: 0.5


  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
save_model(best_model['RNN'][1], "model/Infected/RNN.pth")
save_hyperparameter(best_model['RNN'][2:], "hyperparameter/Infected/RNN.pkl")

In [ ]:
load_model(model, "model/Infected/RNN.pth")

In [ ]:
plt.plot(loss_list)

In [ ]:
label_y, predicted = predict_mto(model, df, x_ss, y_ms)

In [ ]:
mape(label_y+1, predicted)

In [ ]:
print(mape(label_y+1, predicted))
plotting(label_y, predicted, 800)

In [ ]:
data